In [3]:
import os
import librosa
import numpy as np
from scipy.ndimage import zoom
import tqdm
from sklearn.decomposition import PCA

# .wav 파일이 저장된 폴더
folder_path = './dataset/'

# .wav 파일을 스펙트로그램으로 변환하여 저장할 리스트
spectrograms = []
labels = []

# '0'과 '1' 폴더를 순회
for label in ['0', '1']:
    label_folder_path = os.path.join(folder_path, label)
    print(label)

    # folder_path 내의 모든 파일을 순회
    for file_name in os.listdir(label_folder_path):
        # .wav 파일만 처리
        if file_name.endswith('.wav'):
            # 오디오 파일 읽기
            y, sr = librosa.load(os.path.join(label_folder_path, file_name), sr=16000)

            hop_length = int(sr * 0.010)  # 프레임 크기 (여기서는 10ms)
            n_fft = int(sr * 0.025)  # FFT 크기 (여기서는 25ms)
            spectrogram = np.abs(librosa.stft(y, hop_length=hop_length, n_fft=n_fft))

            # # 오디오 파일을 스펙트로그램으로 변환
            # D = librosa.stft(y)
            # spectrogram = librosa.amplitude_to_db(np.abs(D), ref=np.max)

            # 리스트에 스펙트로그램 추가
            spectrograms.append(spectrogram)

            # 레이블 추가
            labels.append(int(label))

# numpy 배열로 변환
X_train = np.array(spectrograms)
y_train = np.array(labels)

X_train = np.expand_dims(X_train, -1)



print(X_train.shape)
print(y_train.shape)



0
1
(10000, 201, 301, 1)
(10000,)


In [1]:
# import librosa
# import numpy as np

# # 음성 데이터 로드
# audio_path = './output.wav'
# audio, sr = librosa.load(audio_path, sr=16000)  # 원하는 샘플링 레이트로 설정 (여기서는 16kHz)

# # 음성 데이터를 스펙트로그램으로 변환
# hop_length = int(sr * 0.010)  # 프레임 크기 (여기서는 10ms)
# n_fft = int(sr * 0.025)  # FFT 크기 (여기서는 25ms)
# spectrogram = np.abs(librosa.stft(audio, hop_length=hop_length, n_fft=n_fft))

# # 변환된 스펙트로그램 확인
# print(spectrogram.shape)  # (주파수 빈도 수, 시간 프레임 수)


(201, 301)


In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# 데이터셋 로드
# X_train: 학습 데이터 (음성 데이터)
# y_train: 학습 데이터에 대한 레이블 (음성에 해당하는 텍스트)
# X_train = ...
# y_train = ...

# 데이터 전처리
# 필요한 전처리 과정을 수행합니다.
# (예: 오디오 데이터를 스펙트로그램으로 변환, 텍스트를 정수 시퀀스로 변환 등)

# 모델 구성
model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(201,301,1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

# 학습된 모델 저장
model.save('voice_recognition_model.h5')


Epoch 1/5


2024-01-03 17:19:33.554952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - ETA: 0s - loss: 0.2289 - accuracy: 0.9201

2024-01-03 17:19:57.324340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 27s 103ms/step - loss: 0.2289 - accuracy: 0.9201 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 2/5
250/250 [==============================] - 25s 97ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 3/5
250/250 [==============================] - 23s 92ms/step - loss: 7.3473e-04 - accuracy: 1.0000 - val_loss: 6.4428e-04 - val_accuracy: 1.0000
Epoch 4/5
250/250 [==============================] - 24s 96ms/step - loss: 3.5584e-04 - accuracy: 1.0000 - val_loss: 3.0857e-04 - val_accuracy: 1.0000
Epoch 5/5
250/250 [==============================] - 24s 95ms/step - loss: 2.0919e-04 - accuracy: 1.0000 - val_loss: 2.1429e-04 - val_accuracy: 1.0000
